Welcome, to Punisher. After running your strategy, view the live results with Dash

```
$ python -m punisher.charts.dash_charts.dash_record --name default_backtest
```

## Define Strategy

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from punisher.common import *

from punisher.strategies.strategy import Strategy

In [2]:
class SimpleStrategy(Strategy):
    def __init__(self, asset, quantity):
        super().__init__()
        self.asset = asset
        self.quantity = quantity

    def log_all(self, orders, data, ctx, utc):
        self.logger = ctx.logger
        if self.logger is not None:
            self.log_epoch_time(utc)
            self.log_ohlcv(data, self.asset.symbol, ctx.exchange.id)
            self.log_orders(orders)
            self.log_performance(ctx)
            self.log_balance(ctx)
            self.log_positions(ctx)
            self.log_metrics(ctx)

    def handle_data(self, data, ctx):
        new_orders = []
        quantity = .05
        price = data.get('close', self.asset.symbol, ctx.exchange.id)
        current_time = data.get('utc')
        if random.random() > 0.5:
            order = order_manager.build_limit_buy_order(
                ctx.exchange, self.asset, quantity, price, current_time)
            new_orders.append(order)
        else:
            order = order_manager.build_market_sell_order(
                ctx.exchange, self.asset, quantity, current_time)
            new_orders.append(order)

        # Optionally cancel pending orders (LIVE trading)
        #pending_orders = ctx.exchange.fetch_open_orders(asset)
        cancel_ids = []

        # Add Metrics and OHLCV to Record
        self.update_metric('SMA', 5.0, ctx)
        self.update_metric('RSI', 10.0, ctx)
        self.update_ohlcv(data, ctx)
        self.log_all(new_orders, data, ctx, current_time)
        return {
            'new_orders': new_orders,
            'cancel_ids': cancel_ids
        }

## Backtest

### Backtest From Local CSV Files

In [3]:
# Download some data from Binance
exchange_id = ex_cfg.BINANCE
timeframe = Timeframe.THIRTY_MIN
start = datetime.datetime(year=2018, month=1, day=1, hour=0)
end = datetime.datetime(year=2018, month=1, day=2, hour=0)
assets = [Asset(coins.ETH, coins.BTC), Asset(coins.BTC, coins.USDT)]
data_exchange = load_exchange(exchange_id)
ohlcv_feed.download_ohlcv([data_exchange], assets, timeframe, start, end)

Downloading: ETH/BTC
Downloaded rows: 48
Downloading: BTC/USDT
Downloaded rows: 48


In [4]:
# Setup the Feed Based Paper Exchange
cash = 1.0
cash_currency = coins.BTC
balance = Balance(
    cash_currency=cash_currency,
    starting_cash=cash
)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=timeframe
)
portfolio = Portfolio(
    cash_currency=cash_currency,
    starting_balance=balance,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
feed = OHLCVFileFeed(
    exchange_ids=[exchange_id],
    assets=assets,
    timeframe=timeframe,
    start=None, # Usually None for backtest, but its possible to filter the csv
    end=None
)
exchange = load_feed_based_paper_exchange(balance, feed, exchange_id)
strategy = SimpleStrategy(assets[0], quantity=.05)
experiment_name = 'default_backtest'

In [5]:
runner.backtest(experiment_name, exchange, balance, portfolio, feed, strategy)

---------------------------------------
Epoch 1 - Timestep: 2018-01-01T00:00:00
---------------------------------------
OHLCV
    O: 0.0536 | C: 0.0537 | V: 6805.7 | T: 2018-01-01T00:00:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.0000 Total Val: 0.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 1.0, 'used': 0.0, 'total': 0.0}
POSITIONS
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 2 - Timestep: 2018-01-01T00:30:00
---------------------------------------
OHLCV
    O: 0.0537 | C: 0.0538 | V: 6277.8 | T: 2018-01-01T00:30:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.0000 Total Val: 0.0000 PnL: -1.0000 Returns: -1.0000
BALANCE
    BTC - {'free': 1.0, 'used': 0.0, 'total': 0.0}
    ETH - {'free': 0.05, 'used': -0.1, 'total': -0.05}
POSITIONS
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 3 - Timestep: 2

TIME 2018-01-01 00:00:00
Order Failed {
    "id": "ed5d21683eec4ea7902147a462d6e6d0",
    "exchange_id": "binance",
    "exchange_order_id": null,
    "price": 0.0,
    "quantity": 0.05,
    "filled_quantity": 0.0,
    "order_type": "MARKET_SELL",
    "status": "FAILED",
    "created_time": "2018-01-01T00:00:00",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "last_updated_time": "2018-01-01T00:00:00",
    "fee": {},
    "attempts": 0,
    "trades": [],
    "error": {
        "code": "INSUFFICIENT_FUNDS",
        "msg": "Insufficient funds to place order for asset ETH/BTC of cost: 0.00269. BTC balance is: 1.0. ETH balance is 0.0"
    },
    "symbol": "ETH/BTC"
}
BEFORE Portfolio update [{
    "id": "ed5d21683eec4ea7902147a462d6e6d0",
    "exchange_id": "binance",
    "exchange_order_id": null,
    "price": 0.0,
    "quantity": 0.05,
    "filled_quantity": 0.0,
    "order_type": "MARKET_SELL",
    "status": "FAILED",
    "created_time": "2018-01-01T00:0

---------------------------------------
Epoch 13 - Timestep: 2018-01-01T06:00:00
---------------------------------------
OHLCV
    O: 0.0554 | C: 0.0550 | V: 7336.4 | T: 2018-01-01T06:00:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0550 | Quantity: 0.0500
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: -0.0000 Returns: -0.0000
BALANCE
    BTC - {'free': 0.9917910499999999, 'used': 0.0082089499999999996, 'total': 0.99999999999999989}
    ETH - {'free': 0.2, 'used': -0.65, 'total': -0.45}
POSITIONS
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 14 - Timestep: 2018-01-01T06:30:00
---------------------------------------
OHLCV
    O: 0.0550 | C: 0.0553 | V: 5916.9 | T: 2018-01-01T06:30:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0553 | Quantity: 0.0500
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: -0.0000 Returns: -0.0000
BALANCE
    BTC - {'free': 0.98904144999999988, 'used': 0.010958549999999999, 'total': 0.99999999999999989}
    ETH - 

BEFORE Portfolio update []
TIME 2018-01-01 06:00:00
BEFORE Portfolio update []
TIME 2018-01-01 06:30:00
BEFORE Portfolio update []
TIME 2018-01-01 07:00:00
BEFORE Portfolio update []
TIME 2018-01-01 07:30:00
BEFORE Portfolio update []
TIME 2018-01-01 08:00:00
BEFORE Portfolio update []
TIME 2018-01-01 08:30:00
BEFORE Portfolio update []
TIME 2018-01-01 09:00:00
BEFORE Portfolio update []
TIME 2018-01-01 09:30:00
BEFORE Portfolio update []


    RSI: 10.00
---------------------------------------
Epoch 22 - Timestep: 2018-01-01T10:30:00
---------------------------------------
OHLCV
    O: 0.0557 | C: 0.0559 | V: 6135.0 | T: 2018-01-01T10:30:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0559 | Quantity: 0.0500
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 0.98073984999999997, 'used': 0.01926015, 'total': 1.0}
    ETH - {'free': 0.2, 'used': -0.9000000000000002, 'total': -0.7000000000000002}
POSITIONS
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 23 - Timestep: 2018-01-01T11:00:00
---------------------------------------
OHLCV
    O: 0.0558 | C: 0.0561 | V: 6981.1 | T: 2018-01-01T11:00:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0561 | Quantity: 0.0500
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 0.97794530000000002, 'used': 0.0220547, 'total': 1.0}
    ETH - {'free': 0.2, 

...

TIME 2018-01-01 14:30:00
BEFORE Portfolio update []
TIME 2018-01-01 15:00:00
BEFORE Portfolio update []
TIME 2018-01-01 15:30:00
BEFORE Portfolio update []
TIME 2018-01-01 16:00:00
BEFORE Portfolio update []
TIME 2018-01-01 16:30:00
BEFORE Portfolio update []
TIME 2018-01-01 17:00:00
BEFORE Portfolio update []
TIME 2018-01-01 17:30:00
BEFORE Portfolio update []
TIME 2018-01-01 18:00:00
BEFORE Portfolio update []
TIME 2018-01-01 18:30:00
BEFORE Portfolio update []


---------------------------------------
Epoch 39 - Timestep: 2018-01-01T19:00:00
---------------------------------------
OHLCV
    O: 0.0566 | C: 0.0565 | V: 6651.8 | T: 2018-01-01T19:00:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 0.94969195000000017, 'used': 0.050308049999999993, 'total': 1.0000000000000002}
    ETH - {'free': 0.2, 'used': -1.2000000000000004, 'total': -1.0000000000000004}
POSITIONS
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 40 - Timestep: 2018-01-01T19:30:00
---------------------------------------
OHLCV
    O: 0.0565 | C: 0.0565 | V: 5073.4 | T: 2018-01-01T19:30:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 0.94969195000000017, 'used': 0.050308049999999993, 'total': 1.00

TIME 2018-01-01 19:00:00
BEFORE Portfolio update []
TIME 2018-01-01 19:30:00
BEFORE Portfolio update []
TIME 2018-01-01 20:00:00
BEFORE Portfolio update []
TIME 2018-01-01 20:30:00
BEFORE Portfolio update []
TIME 2018-01-01 21:00:00
BEFORE Portfolio update []
TIME 2018-01-01 21:30:00
BEFORE Portfolio update []
TIME 2018-01-01 22:00:00
BEFORE Portfolio update []
TIME 2018-01-01 22:30:00
BEFORE Portfolio update []
TIME 2018-01-01 23:00:00
BEFORE Portfolio update []


    O: 0.0562 | C: 0.0564 | V: 6290.2 | T: 2018-01-01T23:30:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0564 | Quantity: 0.0500
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 0.9412615000000002, 'used': 0.058738499999999992, 'total': 1.0000000000000002}
    ETH - {'free': 0.2, 'used': -1.5000000000000007, 'total': -1.3000000000000007}
POSITIONS
METRICS
    SMA: 5.00
    RSI: 10.00


TIME 2018-01-01 23:30:00
BEFORE Portfolio update []
No data after prior poll: 2018-01-01 23:30:00


In [ ]:
record

### Backtest from External Exchange Data

In [ ]:
cash = 1.0
cash_currency = coins.BTC
balance = Balance(
    cash_currency=cash_currency,
    starting_cash=cash
)
exchange = load_ccxt_based_paper_exchange(balance, ex_cfg.BINANCE)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    cash_currency=cash_currency,
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)

assets = [Asset(coins.ETH, coins.BTC), Asset(coins.LTC, coins.BTC)]
feed = OHLCVExchangeFeed(
    exchanges=[exchange],
    assets=assets,
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow()-datetime.timedelta(days=1),
    end=None
)
strategy = SimpleStrategy(assets[0], quantity=.05)

In [ ]:
experiment_name = 'default_backtest_exchange'
runner.backtest(experiment_name, exchange, balance, portfolio, feed, strategy)

## Simulate Orders

In [ ]:
cash = 1.0
cash_currency = coins.BTC
balance = Balance(
    cash_currency=coins.BTC,
    starting_cash=cash
)
exchange = load_ccxt_based_paper_exchange(balance, ex_cfg.BINANCE)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    cash_currency=cash_currency,
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)

assets = [Asset(coins.ETH, coins.BTC), Asset(coins.LTC, coins.BTC)]
feed = OHLCVExchangeFeed(
    exchanges=[exchange],
    assets=assets,
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow(),
    end=None
)
strategy = SimpleStrategy(assets[0], quantity=.05)

In [ ]:
experiment_name = 'default_simulate'
runner.simulate(experiment_name, exchange, balance, portfolio, feed, strategy)

## Live Trade

In [ ]:
exchange = load_exchange(ex_cfg.BINANCE)
cash = exchange.fetch_balance().get(coins.BTC)[BalanceType.FREE]
cash_currency = coins.BTC
print(coins.BTC, "cash", cash)

In [ ]:
balance = Balance(
    cash_currency=coins.BTC,
    starting_cash=cash
)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    cash_currency=cash_currency,
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
assets = [Asset(coins.ETH, coins.BTC), Asset(coins.LTC, coins.BTC)]
feed = OHLCVExchangeFeed(
    exchanges=[exchange],
    assets=assets,
    timeframe=Timeframe.ONE_MIN,
    start=None,
    end=None
)
experiment_name = 'default_live'

In [ ]:
runner.live(experiment_name, exchange, balance, portfolio, feed, strategy)